# r/feminism Baseline Analysis

These two datasets contain every archived post from r/feminism between June 2017 to the end of 2017.

In [6]:
femPosts <- read.csv("feminism_posts_06.30.2017-12.31.2107.csv")
femComments <- read.csv("feminism_comments_on_posts_with_body_06.30.2017-12.31.2107.csv")

In [13]:
colnames(femComments)

[1] "X"                      "author"                 "author_flair_css_class"
 [4] "author_flair_text"      "body"                   "body.charlength"       
 [7] "controversiality"       "created"                "created_utc"           
[10] "distinguished"          "eligible"               "gilded"                
[13] "id"                     "link_id"                "parent_id"             
[16] "previous.comments"      "previous.posts"         "retrieved_on"          
[19] "score"                  "subreddit"              "subreddit_id"

In [35]:
colnames(femPosts)

[1] "X"                      "archived"               "author"                
 [4] "author_flair_css_class" "author_flair_text"      "created"               
 [7] "created_utc"            "distinguished"          "domain"                
[10] "eligible"               "gilded"                 "hide_score"            
[13] "id"                     "is_self"                "link_flair_css_class"  
[16] "link_flair_text"        "num_comments"           "over_18"               
[19] "permalink"              "previous.comments"      "previous.posts"        
[22] "quarantine"             "retrieved_on"           "score"                 
[25] "selftext"               "stickied"               "subreddit"             
[28] "subreddit_id"           "thumbnail"              "title"                 
[31] "url"

### First question: How many newcomer commenters does the subreddit receive per day?

This depends on how we define a newcomer. For this analysis, I'll assume that we mean 'someone who has never posted or commented before' as a newcomer.

In [14]:
newcomers <- femPosts[femPosts$previous.comments == 0 & femPosts$previous.posts == 0,]
neverComment <- femPosts[femPosts$previous.comments == 0,]

In [15]:
# how many days are represented in our data?
(range <- as.numeric(max(as.Date(newcomers$created)) - min(as.Date(newcomers$created))))

[1] 184

In [16]:
round(nrow(newcomers) / range)    # never comment or post
round(nrow(neverComment) / range) # never comment

[1] 16

[1] 19

We get an average of 16 folks per day who have never posted / commented, and 19 who have never commented before.

### Second question: How many newcomer commenters does the subreddit receive per post?


In [22]:
round(sum(femComments$previous.comments == 0) / nrow(femPosts))

[1] 1

### Third question: How many newcomer posters does the subreddit receive per day?

In [28]:
round(sum(femComments$previous.comments == 0) / range)

[1] 20